In [ ]:
import json
import requests
import boto3
import tweepy 
import pandas as pd
import datetime
from datetime import date
import re
from geotext import GeoText
import pycountry
import os
import csv
import string
import numpy as np
from ttp import ttp
from datetime import datetime
from requests_oauthlib import OAuth1Session
# from ttp import parser 


def lambda_handler(event, context): 

    cons_tok = cons_tok
    cons_sec = cons_key
    app_tok = app_tok
    app_sec = app_sec
    
    auth = tweepy.OAuthHandler(cons_tok,cons_sec)
    auth.set_access_token(app_tok,app_sec)
    
    api = tweepy.API(auth)
    
    #Get twitter information about my twitter account
    user = api.me()
    
    tweets = tweepy.Cursor(api.user_timeline).items(100) 
    tweets_info = []
    for t in tweets:
      tweets_info.append((t.id,
                          t.created_at,
                          t.lang,
                          t.favorite_count,
                          t.favorited,
                          t.retweet_count,
                          t.retweeted,
                          t.in_reply_to_screen_name,
                          t.text))


    activity_data = pd.DataFrame(tweets_info,columns= ['id', 'created_at', 'lang', 'total_likes', 'like_by_user', 'total_retweets', 'retweet_by_user', 'reply_to_tweet','tweet'])
    
    
    activity_data['tweet'] = activity_data['tweet'].str.replace('[^\w\s\n#@/:%.,_-]', '', flags=re.UNICODE)
    activity_data['tweet'] = activity_data['tweet'].astype(str)
    activity_data['tweet'] = activity_data['tweet'].apply(lambda x:x.replace("\n"," ")) 
    activity_data['tweet'] = activity_data.tweet.apply(lambda x:x.replace(","," "))
    activity_data['tweet'] = activity_data.tweet.apply(lambda x:x.strip())
    
    # #fill none values with zeros 
    activity_data["reply_to_tweet"].fillna("0", inplace = True) 
    # counts = {'0' : 0, 'screen_name': 1} 
    reply_to_tweet_by_user = []
    for i in activity_data["reply_to_tweet"]:
        if 'screen_name' in i:
            reply_to_tweet_by_user.append('1')
        else:
            reply_to_tweet_by_user.append('0')  
    
    reply_to_tweet_by_others = []
    for i in activity_data["reply_to_tweet"]:
        if 'screen_name' and '0' not in i:
            reply_to_tweet_by_others.append('1')
        else:
            reply_to_tweet_by_others.append('0')
    
    activity_data['reply_to_tweet_by_user'] = reply_to_tweet_by_user
    activity_data['reply_to_tweet_by_others'] = reply_to_tweet_by_others 
    activity_data.reply_to_tweet_by_user = activity_data.reply_to_tweet_by_user.astype(int)
    activity_data.reply_to_tweet_by_others = activity_data.reply_to_tweet_by_others.astype(int)
    activity_data['engagement'] = activity_data['total_likes'] + activity_data['total_retweets'] + activity_data['reply_to_tweet_by_user'] + activity_data['reply_to_tweet_by_others']
    
    # # values where key matchesreply_to_tweet
    # activity_data.reply_to_tweet_by_user = [counts[item] for item in activity_data.reply_to_tweet_by_user]
    activity_data.like_by_user = activity_data.like_by_user.astype(int)
    activity_data['likes_by_followers'] = activity_data['total_likes'] - activity_data['like_by_user']
    activity_data.retweet_by_user = activity_data.retweet_by_user.astype(int)
    activity_data['retweet_by_followers'] = activity_data['total_retweets'] - activity_data['retweet_by_user']
    activity_data['created_at'] = pd.to_datetime(activity_data['created_at'])
    # activity_data['created_at'] = [datetime.date(d) for d in activity_data['created_at']]# extracting date from timestamp
    now = datetime.now() 
    dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
    activity_data['latest_date_time'] = dt_string
    to_datetime_fmt = pd.to_datetime(activity_data['latest_date_time'],format='%Y-%m-%d %H:%M:%S')
    activity_data['Days_after_tweet'] = to_datetime_fmt.sub(activity_data['created_at'],axis=0)
    
    
    activity_data['created'] = [datetime.date(d) for d in activity_data['created_at']]
    # activity_data['created_at'] = activity_data['created_at'].astype(str)
    # activity_data['created_at'] = pd.to_datetime(activity_data['created_at'].astype(str), format='%Y-%m')
    activity_data.insert(13, "tweet_count", "1")
    

    Tags = [] 
    for i in activity_data['tweet']:
        if len(re.findall(r"#(\w+)", i)) == 0:
            Tags.append('None')
        else:
            Tags.append(re.findall(r"#(\w+)", i))
            
            
    ment = []
    for j in activity_data['tweet']:
        if len(re.findall(r"@(\w+)", j)) == 0:
            ment.append('None')
        else:
            ment.append(re.findall(r"@(\w+)", j))
    
    
    activity_data['hashtags'] = Tags
    activity_data['mentions'] = ment
    
    # # popular hashtags
    hashtags_list_df = activity_data.loc[
                      activity_data.hashtags.apply(
                          lambda hashtags_list: hashtags_list !=[]
                      ),['hashtags']]
                      
    # create dataframe where each use of hashtag gets its own row
    flattened_hashtags_df = pd.DataFrame(
        [hashtag for hashtags_list in hashtags_list_df.hashtags
        for hashtag in hashtags_list],columns=['hashtags'])
    
    # count of appearances of each hashtag
    popular_hashtags = flattened_hashtags_df.groupby('hashtags').size()\
                                        .reset_index(name='counts')\
                                        .sort_values('counts', ascending=False)\
                                        .reset_index(drop=True)
                                        

    popular_hashtags_pd_user = pd.DataFrame(popular_hashtags,columns= ['hashtags','counts']).to_csv(index=False)
    
    # # popular mentions
    mentions_list_df = activity_data.loc[
                      activity_data.mentions.apply(
                          lambda mentions_list: mentions_list !=[] 
                      ),['mentions']]
                      
    # create dataframe where each use of hashtag gets its own row
    flattened_mentions_df = pd.DataFrame(
        [mention for mentions_list in mentions_list_df.mentions
        for mention in mentions_list],columns=['mentions'])
    
    # count of appearances of each hashtag
    popular_mentions = flattened_mentions_df.groupby('mentions').size()\
                                        .reset_index(name='counts')\
                                        .sort_values('counts', ascending=False)\
                                        .reset_index(drop=True)
                                        
    
    popular_mentions_pd_user = pd.DataFrame(popular_mentions,columns= ['mentions','counts']).to_csv(index=False)
    
    tweets_data = pd.DataFrame(activity_data,columns= ['id', 'created','Days_after_tweet', 'lang','engagement', 'total_likes', 'like_by_user', 'total_retweets', 'retweet_by_user', 'reply_to_tweet_by_user','reply_to_tweet_by_others', 'likes_by_followers', 'retweet_by_followers','tweet_count','tweet'])#.to_csv(index=False)
    
    mask = (tweets_data['Days_after_tweet'] > '0 days 00:00:00') & (tweets_data['Days_after_tweet'] <= '30 days 00:00:00')
    latest_tweets = tweets_data.loc[mask]
    x= latest_tweets[latest_tweets['retweet_by_user']==0]
    y = latest_tweets.reset_index()
    size = 50
    list_of_dfs = [y.loc[i:i+size-1,:] for i in range(0, len(y),size)]
    

    # Make the request
    oauth = OAuth1Session(
        consumer_key ,
        client_secret=consumer_secret,
        resource_owner_key=access_token,
        resource_owner_secret=access_token_secret,
    )
    
    # # Add the Tweet ID for the Tweet you are looking for. You can add up to 50 comma seperated IDs.
    #params = {"ids": "1187283824498987008,1186960396126584832,1186926131338854401,1186926128805511170"}
    id=[]
    count1=[]
    count2=[]
    for k in range(len(list_of_dfs)):
        ids_string=""
        for i in list_of_dfs[k]['id']:
            ids_string+=str(i)+","
        ids_string=ids_string.rstrip("")
        params = {'ids':ids_string}
        response = oauth.get("https://api.twitter.com/labs/1/tweets/metrics/private", params=params)
        a1 = json.loads(response.text)
        for i in range(len(a1["data"])):
            id.append(a1["data"][i]["tweet_id"])
            count1.append(a1["data"][i]["tweet"]["impression_count"])
            if "video" in a1["data"][i]:
                count2.append(a1["data"][i]["video"]["view_count"])
            else:
                count2.append(None)
    
    
    df=pd.DataFrame({"Tweet_id":id,"Impression_count":count1,"view_count":count2}) 
    tweets_data['id'] = tweets_data['id'].astype(str)
    df['id'] = df['Tweet_id']
    result = pd.merge(tweets_data,df,on='id')
    

    tweets_data2 = pd.merge(tweets_data,result[['id','created','Impression_count','view_count']],on=['id','created'],how='outer')#.to_csv(index=False)
    tweets_data2["Impression_count"].fillna('0', inplace = True)
    tweets_data2["view_count"].fillna('0', inplace = True)
    tweets_data2['view_count'] = tweets_data2['view_count'].astype(int)
    tweets_data2["Impression_count"] = tweets_data2["Impression_count"].astype(int)
    tweets_data3 = pd.DataFrame(tweets_data2,columns= ['id', 'created','Days_after_tweet', 'lang','engagement','Impression_count','view_count', 'total_likes', 'like_by_user', 'total_retweets', 'retweet_by_user', 'reply_to_tweet_by_user','reply_to_tweet_by_others', 'likes_by_followers', 'retweet_by_followers','tweet_count','tweet']).to_csv(index=False)
    
    # # # #
    demography_audience = tweepy.Cursor(api.followers, screen_name="screen_name").items(100)
    demography = []
    for d in demography_audience:
        demography.append((d.id,
                      d.created_at,
                      d.screen_name,
                      d.name,
                      d.friends_count,
                      d.followers_count,
                      d.listed_count,
                      d.statuses_count,
                      d.favourites_count,
                      d.following,
                      d.location)) 
                    
                    #   d.description,

    demography_audience_pd = pd.DataFrame(demography, 
                                                columns =['id', 'Account_creation_date', 'screen_name','name',  'friends_count', 'followers_count', 'listed_count', 'total_tweets', 'total_likes_on_tweets', 'followed_by_user','locations'])#'description','name',  .to_csv(index=False)
                                
    
    demography_audience_pd['name'] = demography_audience_pd['name'].str.replace('[^\w\s\n#@/:%.,_-]', '', flags=re.UNICODE)
    demography_audience_pd['name'] = demography_audience_pd['name'].astype(str)
    demography_audience_pd['name'] = demography_audience_pd['name'].apply(lambda x:x.replace("\n"," ")) 
    demography_audience_pd['name'] = demography_audience_pd.name.apply(lambda x:x.replace(","," "))
    demography_audience_pd['name'] = demography_audience_pd.name.apply(lambda x:x.strip())
    
    demography_audience_pd['screen_name'] = demography_audience_pd['screen_name'].str.replace('[^\w\s\n#@/:%.,_-]', '', flags=re.UNICODE)
    demography_audience_pd['screen_name'] = demography_audience_pd['screen_name'].astype(str)
    demography_audience_pd['screen_name'] = demography_audience_pd['screen_name'].apply(lambda x:x.replace("\n"," ")) 
    demography_audience_pd['screen_name'] = demography_audience_pd.screen_name.apply(lambda x:x.replace(","," "))
    demography_audience_pd['screen_name'] = demography_audience_pd.screen_name.apply(lambda x:x.strip())
    

    ## City Name
    city= []
    for i in demography_audience_pd['locations']:
        places = GeoText(i)
        if len(places.cities) == 0:
            city.append('None')
        else:
            city.append(places.cities[0])
            
    
    ## Country Name        
    country_name =[]
    country_list = [i.name for i in list(pycountry.countries)]

    for j in demography_audience_pd['locations'].to_list():
        k = 0
        for i in country_list:
            if  i in j:
                country_name.append(i)
            else:
                k= k+1
        
        if k==249:
            country_name.append('None')
            
    
     
    demography_audience_pd['cities'] = city
    demography_audience_pd['country_name'] = country_name
    demography_audience_pd["screen_name"].fillna('None', inplace = True)
    demography_audience_pd["name"].fillna('None', inplace = True)
    demography_audience_pd["friends_count"].fillna('0', inplace = True)
    demography_audience_pd["friends_count"] = demography_audience_pd["friends_count"].astype(int)
    demography_audience_pd["followers_count"].fillna('0', inplace = True)
    demography_audience_pd["followers_count"] = demography_audience_pd["followers_count"].astype(int)
    demography_audience_pd["listed_count"].fillna('0', inplace = True)
    demography_audience_pd["listed_count"] = demography_audience_pd["listed_count"].astype(int)
    demography_audience_pd["total_tweets"].fillna('0', inplace = True)
    demography_audience_pd["total_tweets"] = demography_audience_pd["total_tweets"].astype(int)
    demography_audience_pd["total_likes_on_tweets"].fillna('0', inplace = True)
    demography_audience_pd["total_likes_on_tweets"] = demography_audience_pd["total_likes_on_tweets"].astype(int)
    # screen_name,name, friends_count, followers_count, listed_count, total_tweets, total_likes_on_tweets
    # demography_audience_pd.insert(12, "count", "1")
    
    # datetime object containing current date and time
    now = datetime.now()
    dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
    demography_audience_pd['latest_date_time'] = dt_string
    to_datetime_fmt = pd.to_datetime(demography_audience_pd['latest_date_time'],format='%Y-%m-%d %H:%M:%S')
    demography_audience_pd['Days_after_join'] = to_datetime_fmt.sub(demography_audience_pd['Account_creation_date'],axis=0)
    demography_audience_pd['Months_after_join'] = (demography_audience_pd['Days_after_join'] / np.timedelta64(1, 'M')).astype(int)
    demography_audience_pd['avg_tweet_count_per_month'] = demography_audience_pd['total_tweets'].divide(demography_audience_pd['Months_after_join'].replace(0,1))
    demography_audience_pd['avg_tweet_likes_per_month'] = demography_audience_pd['total_likes_on_tweets'].divide(demography_audience_pd['Months_after_join'].replace(0,1))
    demography_audience_pd["avg_tweet_count_per_month"].fillna('0', inplace = True)
    demography_audience_pd["avg_tweet_count_per_month"] = demography_audience_pd["avg_tweet_count_per_month"].astype(int)
    demography_audience_pd['Account_creation_date'] = pd.to_datetime(demography_audience_pd['Account_creation_date'])
    demography_audience_pd['Account_creation_date'] = [datetime.date(d) for d in demography_audience_pd['Account_creation_date']]
    
    demography_audience = pd.DataFrame(demography_audience_pd,
                                                columns =['id', 'Account_creation_date', 'screen_name','name',  'friends_count', 'followers_count', 'listed_count', 'total_tweets', 'total_likes_on_tweets', 'followed_by_user','cities','country_name','Days_after_join','Months_after_join','years_after_join', 'avg_tweet_count_per_month','avg_tweet_likes_per_month']).to_csv(index=False)
     
    
                                                
    # # # # 
    demography_friends = tweepy.Cursor(api.friends, screen_name="screen_name").items(100)
    demography_friend = []
    for f in demography_friends:
        demography_friend.append((f.id,
                      f.created_at,
                      f.screen_name,
                      f.name,
                      #f.description,
                      f.friends_count,
                      f.followers_count,
                      f.listed_count,
                      f.statuses_count,
                      f.favourites_count,
                      f.following,
                      f.location))
                       
                       

    demography_friends_pd = pd.DataFrame(demography_friend,
                                                columns =['id', 'Account_creation', 'screen_name', 'name','friends_count', 'followers_count', 'listed_count', 'total_tweets', 'total_likes_on_tweets', 'followed_by_user', 'locations'])#,'description',.to_csv(index=False)
    
    
    demography_friends_pd['name'] = demography_friends_pd['name'].str.replace('[^\w\s\n#@/:%.,_-]', '', flags=re.UNICODE)
    demography_friends_pd['name'] = demography_friends_pd['name'].astype(str)
    demography_friends_pd['name']= demography_friends_pd['name'].apply(lambda x:x.replace("\n"," ")) 
    demography_friends_pd['name']= demography_friends_pd.name.apply(lambda x:x.replace(","," "))
    demography_friends_pd['name'] = demography_friends_pd.name.apply(lambda x:x.strip())
    
    demography_friends_pd['screen_name'] = demography_friends_pd['screen_name'].str.replace('[^\w\s\n#@/:%.,_-]', '', flags=re.UNICODE)
    demography_friends_pd['screen_name'] = demography_friends_pd['screen_name'].astype(str)
    demography_friends_pd['screen_name']= demography_friends_pd['screen_name'].apply(lambda x:x.replace("\n"," ")) 
    demography_friends_pd['screen_name']= demography_friends_pd.screen_name.apply(lambda x:x.replace(","," "))
    demography_friends_pd['screen_name'] = demography_friends_pd.screen_name.apply(lambda x:x.strip())

    
    ## City Friends
    city_friends= []
    for i in demography_friends_pd['locations']:
        places = GeoText(i)
        if len(places.cities) == 0:
            city_friends.append('None')
        else:
            city_friends.append(places.cities[0])
            
            
    ## Country Friends
    country_name_friends =[]
    country_list = [i.name for i in list(pycountry.countries)]

    for j in demography_friends_pd['locations'].to_list():
        k = 0
        for i in country_list:
            if  i in j:
                country_name_friends.append(i)
            else:
                k= k+1
        
        if k==249:
            country_name_friends.append('None')
            
            
    demography_friends_pd['cities'] = city_friends
    demography_friends_pd["screen_name"].fillna('None', inplace = True) 
    demography_friends_pd["name"].fillna('None', inplace = True)
    demography_friends_pd["friends_count"].fillna('0', inplace = True)
    demography_friends_pd["friends_count"] = demography_friends_pd["friends_count"].astype(int)
    demography_friends_pd["followers_count"].fillna('0', inplace = True)
    demography_friends_pd["followers_count"] = demography_friends_pd["followers_count"].astype(int)
    demography_friends_pd["listed_count"].fillna('0', inplace = True)
    demography_friends_pd["listed_count"] = demography_friends_pd["listed_count"].astype(int)
    demography_friends_pd["total_tweets"].fillna('0', inplace = True)
    demography_friends_pd["total_tweets"] = demography_friends_pd["total_tweets"].astype(int)
    demography_friends_pd["total_likes_on_tweets"].fillna('0', inplace = True)
    demography_friends_pd["total_likes_on_tweets"] = demography_friends_pd["total_likes_on_tweets"].astype(int)
    
    # datetime object containing current date and time
    now = datetime.now()
    dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
    demography_friends_pd['latest_date_time'] = dt_string
    to_datetime_fmt = pd.to_datetime(demography_friends_pd['latest_date_time'],format='%Y-%m-%d %H:%M:%S')
    demography_friends_pd['Days_after_join'] = to_datetime_fmt.sub(demography_friends_pd['Account_creation'],axis=0)
    demography_friends_pd['Months_after_join'] = (demography_friends_pd['Days_after_join'] / np.timedelta64(1, 'M')).astype(int)
    demography_friends_pd['years_after_join'] = (demography_friends_pd['Days_after_join'] / np.timedelta64(1, 'Y')).astype(int)
    demography_friends_pd['avg_tweet_count_per_month'] = demography_friends_pd['total_tweets'].divide(demography_friends_pd['Months_after_join'].replace(0,1))
    demography_friends_pd['avg_tweet_likes_per_month'] = demography_friends_pd['total_likes_on_tweets'].divide(demography_friends_pd['Months_after_join'].replace(0,1))
    demography_friends_pd["avg_tweet_count_per_month"].fillna('0', inplace = True)
    demography_friends_pd["avg_tweet_count_per_month"] = demography_friends_pd["avg_tweet_count_per_month"].astype(int)
    demography_friends_pd['Account_creation'] = pd.to_datetime(demography_friends_pd['Account_creation'])
    demography_friends_pd['Account_creation'] = [datetime.date(d) for d in demography_friends_pd['Account_creation']]



    demography_friends = pd.DataFrame(demography_friends_pd, columns =['id', 'Account_creation', 'screen_name','name','friends_count', 'followers_count', 'listed_count', 'total_tweets', 'total_likes_on_tweets', 'followed_by_user','cities','Days_after_join','Months_after_join','years_after_join', 'avg_tweet_count_per_month','avg_tweet_likes_per_month']).to_csv(index=False)
        
    
    
    ## Mentions
    mentioned_tweets = api.mentions_timeline(screen_name = 'screen_name',count=200,tweet_mode="extended")
    mentions = [[tweet.id,tweet.created_at, tweet.lang, tweet.favorite_count, tweet.retweet_count,tweet.full_text] for tweet in mentioned_tweets]

    mentions_pd = pd.DataFrame(mentions,columns=['id', 'dates', 'language','likes', 'retweets','tweets'])
    
    mentions_pd['tweets'] = mentions_pd['tweets'].str.replace('[^\w\s\n#@/:%.,_-]', '', flags=re.UNICODE)
    mentions_pd['tweets'] = mentions_pd['tweets'].astype(str)
    mentions_pd['tweets'] = mentions_pd['tweets'].apply(lambda x:x.replace("\n"," ")) 
    mentions_pd['tweets'] = mentions_pd.tweets.apply(lambda x:x.replace(","," "))
    mentions_pd['tweets'] = mentions_pd.tweets.apply(lambda x:x.strip())
                            
    mentions_pd['dates'] = pd.to_datetime(mentions_pd['dates'])
    mentions_pd['dates'] = [datetime.date(d) for d in mentions_pd['dates']]# extracting date from timestamp
    mentions_pd['dates'] = mentions_pd['dates'].astype(str)
    mentions_pd['engagement'] = mentions_pd['likes'] + mentions_pd['retweets']
    mentions_pd.insert(4, "tweet_count", "1")
    

    Tag = []
    for i in mentions_pd['tweets']:
        if len(re.findall(r"#(\w+)", i)) == 0:
            Tag.append('None')
        else:
            Tag.append(re.findall(r"#(\w+)", i))
            
            
    men = []
    for j in mentions_pd['tweets']:
        if len(re.findall(r"@(\w+)", j)) == 0:
            men.append('None')
        else:
            men.append(re.findall(r"@(\w+)", j))
    
    
    mentions_pd['hashtags'] = Tag
    mentions_pd['mentions'] = men
    
    # # popular hashtags
    hashtags_list_df = mentions_pd.loc[
                      mentions_pd.hashtags.apply(
                          lambda hashtags_list: hashtags_list !=[]
                      ),['hashtags']]
                      
    # create dataframe where each use of hashtag gets its own row
    flattened_hashtags_df = pd.DataFrame(
        [hashtag for hashtags_list in hashtags_list_df.hashtags
        for hashtag in hashtags_list],columns=['hashtags'])
    
    # count of appearances of each hashtag
    popular_hashtags = flattened_hashtags_df.groupby('hashtags').size()\
                                        .reset_index(name='counts')\
                                        .sort_values('counts', ascending=False)\
                                        .reset_index(drop=True)
                                        

    popular_hashtags_pd_mentions = pd.DataFrame(popular_hashtags,columns= ['hashtags','counts']).to_csv(index=False)
    
    # popular mentions
    mentions_list_df = mentions_pd.loc[
                      mentions_pd.mentions.apply(
                          lambda mentions_list: mentions_list !=[]
                      ),['mentions']]
                      
    # create dataframe where each use of hashtag gets its own row
    flattened_mentions_df = pd.DataFrame(
        [mention for mentions_list in mentions_list_df.mentions
        for mention in mentions_list],columns=['mentions'])
    
    # count of appearances of each hashtag
    popular_mentions = flattened_mentions_df.groupby('mentions').size()\
                                        .reset_index(name='counts')\
                                        .sort_values('counts', ascending=False)\
                                        .reset_index(drop=True)
                                        
    
    popular_mentions_pd_mentions = pd.DataFrame(popular_mentions,columns= ['mentions','counts']).to_csv(index=False)
    
    
    mentioned_data = pd.DataFrame(mentions_pd,columns=['id', 'dates','language','likes', 'retweets','engagement', 'tweet_count','tweets']).to_csv(index=False)
    
    
    client=boto3.client("ec2")
    s3 = boto3.client('s3')
    s3.put_object(Bucket="yalla",Key="data/twitter/tweet_activity/tweet_activity.csv",Body=str(tweets_data3))
    s3.put_object(Bucket="yalla",Key="data/twitter/frequent_hashtags_user/frequent_hashtags_user.csv",Body=str(popular_hashtags_pd_user))
    s3.put_object(Bucket="yalla",Key="data/twitter/frequent_mentions_user/frequent_mentions_user.csv",Body=str(popular_mentions_pd_user))
    s3.put_object(Bucket="yalla",Key="data/twitter/followers_info/followers_info.csv",Body=str(demography_audience))
    s3.put_object(Bucket="yalla",Key="data/twitter/friends_info/friends_info.csv",Body=str(demography_friends))
    s3.put_object(Bucket="yalla",Key="data/twitter/mentioned_tweets/mentioned_tweets.csv",Body=str(mentioned_data))
    s3.put_object(Bucket="yalla",Key="data/twitter/popular_hashtags_mentions/popular_hashtags_mentions.csv",Body=str(popular_hashtags_pd_mentions))
    s3.put_object(Bucket="yalla",Key="data/twitter/popular_mentions_mentions/popular_mentions_mentions.csv",Body=str(popular_mentions_pd_mentions))

     
    
    s3_re = boto3.resource('s3')  
    bucket="yalla"
    prefix_1 = "data/twitter/tweet_activity"
    prefix_2 = "data/twitter/frequent_hashtags_user"
    prefix_3 = "data/twitter/frequent_mentions_user"
    prefix_4 = "data/twitter/followers_info"
    prefix_5 = "data/twitter/friends_info" 
    prefix_6 = "data/twitter/mentioned_tweets"
    prefix_7 = "data/twitter/popular_hashtags_mentions"
    prefix_8 = "data/twitter/popular_mentions_mentions" 


    s3_re.Object(bucket, prefix_1+"/{}.csv".format(str(datetime.now()))).put(Body=str(tweets_data3))
    s3_re.Object(bucket, prefix_2+"/{}.csv".format(str(datetime.now()))).put(Body=str(popular_hashtags_pd_user))
    s3_re.Object(bucket, prefix_3+"/{}.csv".format(str(datetime.now()))).put(Body=str(popular_mentions_pd_user)) 
    s3_re.Object(bucket, prefix_4+"/{}.csv".format(str(datetime.now()))).put(Body=str(demography_audience))
    s3_re.Object(bucket, prefix_5+"/{}.csv".format(str(datetime.now()))).put(Body=str(demography_friends))
    s3_re.Object(bucket, prefix_6+"/{}.csv".format(str(datetime.now()))).put(Body=str(mentioned_data))
    s3_re.Object(bucket, prefix_7+"/{}.csv".format(str(datetime.now()))).put(Body=str(popular_hashtags_pd_mentions))
    s3_re.Object(bucket, prefix_8+"/{}.csv".format(str(datetime.now()))).put(Body=str(popular_mentions_pd_mentions))

    
    